# Reverse Geocoding

Reverse geocoding is the process to convert the latitude and longitude coordinates to a readable address.

![reverse_geocode](images/reverse_geocode.png)

Let us try reverse geocoding our starbucks dataset with in  5 miles of Case Western Reserve University. 

So let us create a geodataframe for Case Western Reserve University

In [1]:
import geopandas as gpd
import pandas as pd

In [4]:
data = pd.DataFrame([['cwru',41.504,-81.608]],columns=['name','lat','lon'])
cwru = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon,data.lat),crs='EPSG:4326')
cwru

,name,lat,lon,geometry
0,cwru,41.504,-81.608,POINT (-81.60800 41.50400)


Now let us create a buffer of  miles around cwru

In [16]:
cwru_buffer = cwru.to_crs('EPSG:26917').buffer(5*1609.34)
cwru_buffer = cwru_buffer.to_crs('EPSG:4326')
cwru_buffer

0    POLYGON ((-81.51159 41.50447, -81.51200 41.497...
dtype: geometry

Now let us load the starbucks data

In [19]:
starbucks = gpd.read_file('../../largedatasets/starbucks/starbucks.shp')
starbucks.head()

,pk_uid,geometry
0,1,POINT (-73.02315 40.82713)
1,2,POINT (-72.99688 40.82229)
2,3,POINT (-73.03259 40.77785)
3,4,POINT (-73.05505 40.86302)
4,5,POINT (-73.09551 40.78991)


In [20]:
starbucks.crs = cwru_buffer.crs

In [21]:
starbucksCWRU = starbucks[starbucks.within(cwru_buffer.geometry.values[0])]
starbucksCWRU

,pk_uid,geometry
5345,5346,POINT (-81.69834 41.49908)
5347,5348,POINT (-81.69477 41.50174)
5348,5349,POINT (-81.68142 41.50056)
7969,7970,POINT (-81.62175 41.50156)
8000,8001,POINT (-81.61756 41.50206)
8001,8002,POINT (-81.62066 41.50210)
8002,8003,POINT (-81.59411 41.50124)
8003,8004,POINT (-81.60588 41.50816)
8005,8006,POINT (-81.60444 41.51258)
8006,8007,POINT (-81.53841 41.50178)


Now let us use reverse geocode to get the addresses

In [26]:
addresses = gpd.tools.reverse_geocode(starbucksCWRU.geometry.values,provider='nominatim',user_agent='test')

In [27]:
addresses

,geometry,address
0,POINT (-81.69854 41.49921),"Johnson Block, West 6th Street, Warehouse Dist..."
1,POINT (-81.69491 41.50212),"Global Center for Health Innovation, 133, Sain..."
2,POINT (-81.68113 41.50056),"Hanna Building, Euclid Avenue, Playhouse Squar..."
3,POINT (-81.61982 41.50217),"Cleveland Clinic, 9500, Euclid Avenue, East Ei..."
4,POINT (-81.61982 41.50217),"Cleveland Clinic, 9500, Euclid Avenue, East Ei..."
5,POINT (-81.62066 41.50206),"(Q) Glickman Tower, East 96th Street, East Eig..."
6,POINT (-81.59366 41.50128),"Heights Center Building, 12417, Cedar Road, Ce..."
7,POINT (-81.60594 41.50819),"Starbucks, Euclid Avenue, Uptown, Cleveland, C..."
8,POINT (-81.60442 41.51280),"House 4, East 115th Street, Uptown, Cleveland,..."
9,POINT (-81.53835 41.50176),"Starbucks, 13939, Cedar Road, South Euclid, Cu..."
